<a href="https://colab.research.google.com/github/sime1/notebooks/blob/master/Ames_Housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ames Housing Dataset

I questo notebook provo ad applicare le tecniche per il preprocessing dei dati al dataset **Ames, IA Real Estate Data**. Questo dataset è stato creato come alternativa al **Boston Housing Dataset**, con più variabili e osservazioni.

Le variabili descrivono un abitazione in vendita in Ames (Iowa), e viene chiesto di prevedere il prezzo; si tratta quindi di un problema di regressione.

Una descrizione completa del dataset può essere trovata all'indirizzo http://jse.amstat.org/v19n3/decock.pdf

## Descrizione delle variabili

La descrizione del dataset, inclusa la descrizione delle singole variabili, è fornita come un file di test insieme al file dati

In [1]:
import numpy as np
import pandas as pd
import urllib
from sklearn.model_selection import train_test_split

descr = urllib.request.urlopen("http://jse.amstat.org/v19n3/decock/DataDocumentation.txt").read().decode('cp1252')
print(descr)

NAME: AmesHousing.txt
TYPE: Population
SIZE: 2930 observations, 82 variables
ARTICLE TITLE: Ames Iowa: Alternative to the Boston Housing Data Set

DESCRIPTIVE ABSTRACT: Data set contains information from the Ames Assessor’s Office used in computing assessed values for individual residential properties sold in Ames, IA from 2006 to 2010.

SOURCES: 
Ames, Iowa Assessor’s Office 

VARIABLE DESCRIPTIONS:
Tab characters are used to separate variables in the data file. The data has 82 columns which include 23 nominal, 23 ordinal, 14 discrete, and 20 continuous variables (and 2 additional observation identifiers).

Order (Discrete): Observation number

PID (Nominal): Parcel identification number  - can be used with city web site for parcel review. 

MS SubClass (Nominal): Identifies the type of dwelling involved in the sale.	

       020	1-STORY 1946 & NEWER ALL STYLES
       030	1-STORY 1945 & OLDER
       040	1-STORY W/FINISHED ATTIC ALL AGES
       045	1-1/2 STORY - UNFINISHED ALL AGES
   

Da questa descrizione si può capire la tipologia delle singole variabili. 

## Preparazione dei dati

Per la preparazione dei dati utilizzo la libreria `pandas` per manipolare le colonne, ed alcuni strumenti forniti da `sklearn` per l'encoding dei dati e la gestione dei dati mancanti.

Come prima cosa elimino le colonne `Order` e `PID`, e separo le feature dai risultati.

In [2]:
dataset = pd.read_csv("http://jse.amstat.org/v19n3/decock/AmesHousing.txt", sep='\t')
print(dataset.shape)
print(dataset.columns)

X, y = dataset.drop(["SalePrice", "Order", "PID"],axis=1), dataset["SalePrice"].values

(2930, 82)
Index(['Order', 'PID', 'MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area',
       'Street', 'Alley', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add', 'Roof Style',
       'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type',
       'Mas Vnr Area', 'Exter Qual', 'Exter Cond', 'Foundation', 'Bsmt Qual',
       'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin SF 1',
       'BsmtFin Type 2', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF',
       'Heating', 'Heating QC', 'Central Air', 'Electrical', '1st Flr SF',
       '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Bsmt Full Bath',
       'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr',
       'Kitchen AbvGr', 'Kitchen Qual', 'TotRms AbvGrd', 'Functional',
       'Fireplaces', 'Fireplace Qu', 'Garage Type', 'Garage Yr B

Utilizzo `OneHotEncoder` per il l'encoding delle feature categoriche nominali, mentre uso `OrdinalEncoder` per le feature categoriche ordinali. Questo permette di ignorare oppure tenere conto dell'ordine delle categorie nei due diversi casi.

Data la loro natura, per le feature categoriche ordinali è necessario specificare l'ordine delle categorie. Per quelle nominali invece è necessario unicamente fornire l'elenco.

Oltre all'encoding delle categorie, il preprocessing include anche dei `SimpleImputer`, che sostituiscono i valori mancanti (`NaN`) con valori di default; nel caso delle categorie viene creata una categoria aggiuntiva (informazione non disponibile), nel caso delle feature numeriche uso la strategia di default (valore medio).

Infine, è importante fare attenzione a fare il fit della pipeline di preprocessing solo sui dati di training, per evitare data leakage.

In [0]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer

nom_names = [
  'MS SubClass',
  'MS Zoning',
  'Street',
  'Alley',
  'Land Contour',
  'Lot Config',
  'Neighborhood',
  'Condition 1',
  'Condition 2',
  'Bldg Type',
  'House Style',
  'Roof Style',
  'Roof Matl',
  'Exterior 1st',
  'Exterior 2nd',
  'Mas Vnr Type',
  'Foundation',
  'Heating',
  'Central Air',
  'Garage Type',
  'Misc Feature',
  'Sale Type',
  'Sale Condition'
]

ordinal = [
  ('Lot Shape', ['Reg', 'IR1', 'IR2', 'IR3']),
  ('Utilities', ['AllPub', 'NoSewr', 'NoSeWa', 'ELO']),
  ('Land Slope', ['Gtl', 'Mod', 'Sev']),
  ('Overall Qual', [x for x in range(10,0,-1)]),
  ('Overall Cond', [x for x in range(10,0,-1)]),
  ('Exter Qual', ['Ex', 'Gd', 'TA', 'Fa', 'Po']),
  ('Exter Cond', ['Ex', 'Gd', 'TA', 'Fa', 'Po']),
  ('Bsmt Qual', ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA']),
  ('Bsmt Cond', ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA']),
  ('Bsmt Exposure', ['Gd', 'Av', 'Mn', 'No', 'NA']),
  ('BsmtFin Type 1', ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'NA']),
  ('BsmtFin Type 2', ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'NA']),
  ('Heating QC', ['Ex', 'Gd', 'TA', 'Fa', 'Po']),
  ('Electrical', ['SBrkr', 'FuseA', 'FuseF', 'FuseP', 'Mix', 'NA']),
  ('Kitchen Qual', ['Ex', 'Gd', 'TA', 'Fa', 'Po']),
  ('Functional', ['Typ', 'Min1', 'Min2', 'Mod', 'Maj1', 'Maj2', 'Sev', 'Sal']),
  ('Fireplace Qu', ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA']),
  ('Garage Finish', ['Fin', 'RFn', 'Unf', 'NA']),
  ('Garage Qual', ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA']),
  ('Garage Cond', ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA']),
  ('Paved Drive', ['Y', 'P', 'N']),
  ('Pool QC', ['Ex', 'Gd', 'TA', 'Fa', 'NA']),
  ('Fence', ['GdPrv', 'MnPrv', 'GdWo', 'MnWw', 'NA'])
]

# creo gli elenchi delle categorie nominali, eventualmente aggiungendo una categoria
# per dati mancanti
nom_cats = [ dataset[cat].unique().astype(object) if not dataset[cat].isnull().any() else 
             np.concatenate((dataset[cat].unique().astype(object), ['NA']))
             for cat in nom_names ]

ord_names, ord_cats = zip(*ordinal)
ord_names, ord_cats = list(ord_names), list(ord_cats)

cat_columns = nom_names + ord_names
quant_columns = [x for x in X.columns if x not in cat_columns]
X = dataset[quant_columns + cat_columns]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.3)

num_col_end = len(quant_columns)
nom_col_end = num_col_end + len(nom_names)
ord_col_end = nom_col_end + len(ordinal)

preprocess_pipeline = make_pipeline(
    ColumnTransformer([
      ("num", 'passthrough', [x for x in range(num_col_end)]),
      ("imp_cat", SimpleImputer(strategy='constant',fill_value='NA'), [x for x in range(num_col_end,ord_col_end)])
    ]),
    ColumnTransformer([
      ("imp_num", SimpleImputer(), [x for x in range(num_col_end)]),
      ("nominal", OneHotEncoder(categories=nom_cats), [x for x in range(num_col_end, nom_col_end)]),
      ("ordinal", OrdinalEncoder(categories=ord_cats), [x for x in range(nom_col_end, ord_col_end)]),
    ])
)


## AdaBoost e Decision Tree

Ho deciso di approfittare di questo dataset per provare a confrontare un *decision tree* con un *ensemble* di *decision tree* creata con l'algoritmo *AdaBoost*.

In entrambi i casi utilizzo `GridSearchCV` per la scelta degli iperparametri del modello; siccome `AdaBoostRegressor` impiega decisamente più tempo per il training, ho deciso di limitare il numero di parametri del base_estimator, mentre per `DecisionTreeRegressor` 

In [4]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.base import clone

params = {
    'decisiontreeregressor__splitter': ['best', 'random'],
    'decisiontreeregressor__max_depth': range(12, 25),
    'decisiontreeregressor__max_features': ['sqrt', 'auto', 'log2', 0.25,  0.5, 0.75]
}

model = make_pipeline(
    clone(preprocess_pipeline),
    DecisionTreeRegressor()
)

DecisionTreeRegressor()

sel = GridSearchCV(model, params, cv=5)

sel.fit(X_train, y_train)
print(f'RegressionTree score:{sel.best_estimator_.score(X_test, y_test)}')
print(sel.best_estimator_.steps[1][1].get_params())

RegressionTree score:0.7789424123264295
{'criterion': 'mse', 'max_depth': 20, 'max_features': 0.5, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'presort': False, 'random_state': None, 'splitter': 'best'}


In [5]:
from sklearn.ensemble import AdaBoostRegressor

tree = DecisionTreeRegressor()
model = make_pipeline(
    clone(preprocess_pipeline),
    AdaBoostRegressor(base_estimator=tree)
)

params = {
    'adaboostregressor__base_estimator__max_depth': range(12,22,3),
    'adaboostregressor__n_estimators': [150],
    'adaboostregressor__loss': ['linear', 'square', 'exponential']
}

sel = GridSearchCV(model, params, cv=5)

sel.fit(X_train, y_train)
print(f'AdaBoost score:{sel.best_estimator_.score(X_test, y_test)}')
print(sel.best_estimator_.steps[1][1].get_params())

AdaBoost score:0.9216424870641478
{'base_estimator__criterion': 'mse', 'base_estimator__max_depth': 12, 'base_estimator__max_features': None, 'base_estimator__max_leaf_nodes': None, 'base_estimator__min_impurity_decrease': 0.0, 'base_estimator__min_impurity_split': None, 'base_estimator__min_samples_leaf': 1, 'base_estimator__min_samples_split': 2, 'base_estimator__min_weight_fraction_leaf': 0.0, 'base_estimator__presort': False, 'base_estimator__random_state': None, 'base_estimator__splitter': 'best', 'base_estimator': DecisionTreeRegressor(criterion='mse', max_depth=12, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best'), 'learning_rate': 1.0, 'loss': 'square', 'n_estimators': 150, 'random_state': None}


### Note

* Per i valori mancanti ho usato direttamente i `SimpleImputer`. Sarebbe stato possibile anche individuare le righe con  i dati mancanti e nel caso fossero poche eliminarle. Ad esempio, se per una feature categorica una sola riga ha informazioni mancanti, può essere meglio eliminare la riga piuttosto che aggiungere una categoria in più per i dati mancanti.
* Il documento di presentazione del dataset suggerisce anche altri passaggi di preparazione, come la rimozione di outlier e l'utilizzo di una [mappa della zona](http://jse.amstat.org/v19n3/decock/AmesResidential.pdf)